# Oasis FM Testing Tool

This notebook allows example insurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included

The following terminology is defined in OED. Please visit https://github.com/Simplitium/OED for more information.



## Input files 

Files must be named the following:

* account.csv
* location.csv
* ri_info.csv
* ri_scope.csv


In [1]:
# 1. Notebook and Python setup
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os
import shutil

# 3rd party Python libraries
import widgets
import functions
import pandas as pd
from IPython.display import Markdown, display, Javascript
from oasislmf.manager import OasisManager


In [2]:
# Upload local files into the Notebook (Optional)

widgets.file_uploader('examples/uploaded')

FileUploadWidget(label='Upload .CSV file', _dom_classes=('widget_item', 'btn-group'))

In [3]:
# Select a validation example to run, the default value points the file upload location
# If a variables shows a 'None' results then the s

source_exposure = {}
widgets.select_source_dir(source_exposure)

'Default value: examples/uploaded'

{'source_dir': 'examples/uploaded',
 'location_path': None,
 'account_path': None,
 'ri_info_path': None,
 'ri_scope_path': None}

'examples/CX30_100'

{'source_dir': 'examples/CX30_100',
 'location_path': 'examples/CX30_100/location.csv',
 'account_path': 'examples/CX30_100/account.csv',
 'ri_info_path': 'examples/CX30_100/ri_info.csv',
 'ri_scope_path': 'examples/CX30_100/ri_scope.csv'}

<IPython.core.display.Javascript object>

In [4]:
# Load source files from selected test case 
location_df =  functions.load_df(source_exposure['location_path'], required_file='location.csv')
account_df =  functions.load_df(source_exposure['account_path'], required_file='account.csv')
ri_info_df =  functions.load_df(source_exposure['ri_info_path'])
ri_scope_df =  functions.load_df(source_exposure['ri_scope_path'])




## Set Test Parameters 

In [5]:
# Keys data 
number_of_subperils = 1
supported_coverage_types = [1,2,3,4]

## Edit Params 
loss_factors = [0.5, 1]
alloc_il = 2
alloc_ri = 3
output_level = 'item' # valid options are 'item', 'loc', 'pol', 'acc', 'port'

In [6]:
# Running this cell will execute the entire notebook 
display(Javascript("Jupyter.notebook.execute_cells_below()"))

<IPython.core.display.Javascript object>

In [7]:
# View/edit the location data.
location_grid = widgets.show_df(location_df)
location_grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [8]:
# View/edit the account data. 
account_grid = widgets.show_df(account_df)
account_grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [9]:
# View/edit the ri_info data (Optinal). 
if ri_info_df is not None:
    ri_info_grid = widgets.show_df(ri_info_df)
    ri_info_grid

In [10]:
# View/edit the ri_scope data (Optinal). 
if ri_scope_df is not None:
    ri_scope_grid = widgets.show_df(ri_scope_df)
    ri_scope_grid

In [11]:
# 6. Pick up any edits in the grid before running the analysis
location_df = location_grid.get_changed_df()
account_df = account_grid.get_changed_df()
ri_info_df = ri_info_grid.get_changed_df()
ri_scope_df = ri_scope_grid.get_changed_df()
 
# Store exposure and create run dir 
run_dir = os.path.join('runs', os.path.basename(source_exposure['source_dir']))
os.makedirs(run_dir, exist_ok=True)

loc_csv = os.path.join(run_dir, "location.csv")
acc_csv = os.path.join(run_dir, "account.csv")
info_csv = os.path.join(run_dir, "ri_info.csv")
scope_csv = os.path.join(run_dir, "ri_scope.csv")

location_df.to_csv(path_or_buf=loc_csv, encoding='utf-8', index=False)
account_df.to_csv(path_or_buf=acc_csv, encoding='utf-8', index=False)
ri_info_df.to_csv(path_or_buf=info_csv, encoding='utf-8', index=False)
ri_scope_df.to_csv(path_or_buf=scope_csv, encoding='utf-8', index=False)

In [12]:
# 7. Generate deterministic keys data 

# Parameters  


# Generate keys file
keys_csv = os.path.join(run_dir, "keys.csv")
OasisManager().generate_keys_deterministic(
    oed_location_csv=loc_csv,
    keys_data_csv=keys_csv,
    supported_oed_coverage_types=supported_coverage_types,
    num_subperils=number_of_subperils,
)

keys_df = functions.load_df(keys_csv, required_file='keys.csv') 
keys_grid = widgets.show_df(keys_df)
keys_grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

## Generate Oasis files

In [13]:
## Generate Oasis files 

# Pick up any edits in the Keys data 
keys_df = keys_grid.get_changed_df()
keys_df.to_csv(path_or_buf=keys_csv, encoding='utf-8', index=False)

# 2. Start Oasis files generation
oasis_files = OasisManager().generate_files(
    oasis_files_dir=run_dir,
    oed_location_csv=loc_csv,
    oed_accounts_csv=acc_csv,
    oed_info_csv=info_csv,
    oed_scope_csv=scope_csv,
    keys_data_csv=keys_csv,
)

In [14]:
# Show FM items 
fm_file = 'items'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


### items.csv

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [15]:
# Show FM coverages 
fm_file = 'coverages'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


### coverages.csv

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [16]:
# Show FM fm_policytc 
fm_file = 'fm_policytc'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


### fm_policytc.csv

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [17]:
# Show FM fm_profile 
fm_file = 'fm_profile'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


### fm_profile.csv

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [18]:
# Show FM fm_programme 
fm_file = 'fm_programme'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


### fm_programme.csv

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

## Generate Losses 


In [19]:
# 3. Run Deterministic Losses


output_losses = os.path.join(run_dir, 'losses.csv')
OasisManager().run_exposure(
    src_dir=run_dir,
    output_level=output_level,
    output_file=output_losses,
    num_subperils=number_of_subperils,
    loss_factor=loss_factors,
    ktools_alloc_rule_il=alloc_il,
    ktools_alloc_rule_ri=alloc_ri,
    net_ri=True,
    include_loss_factor=True,
    print_summary=False,
)
losses = functions.load_df(output_losses)
for i in range(len(loss_factors)):
    factor_losses = losses[losses.loss_factor_idx == i]
    display(Markdown(f'### Loss Factor {loss_factors[i]*100} %'))
    
    display(Markdown('**Total gul** = {}'.format(
        factor_losses.loss_gul.sum()
    )))
    display(Markdown('**Total il** = {}'.format(
        factor_losses.loss_il.sum()
    )))
    display(Markdown('**Total ri ceded** = {}'.format(
        factor_losses.loss_ri.sum()
    )))
    
    
    
    display(factor_losses)


### Loss Factor 50.0 %

**Total gul** = 400000000.0

**Total il** = 343000000.0

**Total ri ceded** = 314599999.0

,output_id,portnumber,accnumber,locnumber,polnumber,coverage_type_id,loss_factor_idx,loss_gul,loss_il,loss_ri
0,1,OEDTest,CX30_08,CX30_08_1,CX30_08_1,1,0,37500000.0,36750000.0,29650000.0
2,2,OEDTest,CX30_08,CX30_08_1,CX30_08_1,3,0,6000000.0,5880000.0,4744000.0
4,3,OEDTest,CX30_08,CX30_08_1,CX30_08_1,4,0,6500000.0,6370000.0,5139333.0
6,4,OEDTest,CX30_08,CX30_08_2,CX30_08_1,1,0,37500000.0,36750000.0,29650000.0
8,5,OEDTest,CX30_08,CX30_08_2,CX30_08_1,3,0,6000000.0,5880000.0,4744000.0
10,6,OEDTest,CX30_08,CX30_08_2,CX30_08_1,4,0,6500000.0,6370000.0,5139333.0
12,7,OEDTest,CX30_08,CX30_08_3,CX30_08_1,1,0,37500000.0,36750000.0,36750000.0
14,8,OEDTest,CX30_08,CX30_08_3,CX30_08_1,3,0,6000000.0,5880000.0,5880000.0
16,9,OEDTest,CX30_08,CX30_08_3,CX30_08_1,4,0,6500000.0,6370000.0,6370000.0
18,10,OEDTest,CX30_08,CX30_08_4,CX30_08_1,1,0,37500000.0,36750000.0,36750000.0


### Loss Factor 100 %

**Total gul** = 800000000.0

**Total il** = 693000000.0

**Total ri ceded** = 0.0

,output_id,portnumber,accnumber,locnumber,polnumber,coverage_type_id,loss_factor_idx,loss_gul,loss_il,loss_ri
1,1,OEDTest,CX30_08,CX30_08_1,CX30_08_1,1,1,75000000.0,74250000.0,0.0
3,2,OEDTest,CX30_08,CX30_08_1,CX30_08_1,3,1,12000000.0,11880000.0,0.0
5,3,OEDTest,CX30_08,CX30_08_1,CX30_08_1,4,1,13000000.0,12870000.0,0.0
7,4,OEDTest,CX30_08,CX30_08_2,CX30_08_1,1,1,75000000.0,74250000.0,0.0
9,5,OEDTest,CX30_08,CX30_08_2,CX30_08_1,3,1,12000000.0,11880000.0,0.0
11,6,OEDTest,CX30_08,CX30_08_2,CX30_08_1,4,1,13000000.0,12870000.0,0.0
13,7,OEDTest,CX30_08,CX30_08_3,CX30_08_1,1,1,75000000.0,74250000.0,0.0
15,8,OEDTest,CX30_08,CX30_08_3,CX30_08_1,3,1,12000000.0,11880000.0,0.0
17,9,OEDTest,CX30_08,CX30_08_3,CX30_08_1,4,1,13000000.0,12870000.0,0.0
19,10,OEDTest,CX30_08,CX30_08_4,CX30_08_1,1,1,75000000.0,74250000.0,0.0
